In [1]:
# 0 = RUNNING
# 1 = STANDING
# 2 = WALKING

In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
# w = window_size
def window(a, w = 5, o = 1, copy = False):
    sh = (a.size - w + 1, w)
    st = a.strides * 2
    view = np.lib.stride_tricks.as_strided(a, strides = st, shape = sh)[0::o]
    if copy:
        return view.copy()
    else:
        return view

In [4]:
def preprocess(filename, label):
    df = pd.read_csv('dataset/'+filename)
    df = df.dropna(axis=0)
    #df = df.sample(frac=1, axis=0).reset_index()
    df_window = pd.DataFrame(np.concatenate((window(df.x1.to_numpy()),window(df.x2.to_numpy()),window(df.x3.to_numpy())),axis=1))
    df_window.columns = [i for i in range(len(df_window.columns))]
    df_window['label'] = label
    return df_window

In [5]:
run = preprocess('running.csv', 0)
stand = preprocess('standing.csv', 1)
walk = preprocess('walking.csv', 2)

In [6]:
run_name = ['Running_1619263170843.csv', 'Running_1619263208673.csv', 'Running_1619263255959.csv', 
            'Running_1619263296112.csv', 'Running_1619263346857.csv', 'Running_below_screen_in.csv', 
            'Running_upper_screen_out.csv', 'Running_1619259552664.csv', 'Running_1619259685377.csv']

stand_name = ['Standing_camdown_screenin_1619283201779.csv', 'Standing_camdown_screenout_1619283090690.csv',
              'Standing_camup_screenin_1619282941930.csv',  'Standing_bottomright_1619357391450.csv',
             'Standing_camup_screenout_1619282750771.csv', 'Standing_bottomleft_1619357289577.csv',
             'Standing_topleft_1619357121075.csv', 'Standing_topright_1619357008320.csv']

walk_name = ['Walking_1619262702376.csv', 'Walking_1619262774656.csv',
             'walking_below_screen_in.csv', 'walking_below_screen_out.csv', 'walking_upper_screen_in.csv',
             'walking_upper_screen_out.csv' , 'Walking_1619259294903.csv', 'Walking_1619259343121.csv',
             'Walking_1619259424085.csv', 'Walking_1619259463954.csv']

In [7]:
for i in run_name:
    f = preprocess(i, 0)
    run = pd.concat((run, f), axis=0).reset_index(drop=True)
for i in stand_name:
    f = preprocess(i, 1)
    stand = pd.concat((stand, f), axis=0).reset_index(drop=True)
for i in walk_name:
    f = preprocess(i, 2)
    walk = pd.concat((walk, f), axis=0).reset_index(drop=True)

In [8]:
print('run',len(run))
print('stand',len(stand))
print('walk',len(walk))

run 8643
stand 10251
walk 11374


In [9]:
xy = pd.concat((run, stand, walk), axis=0).reset_index(drop=True)
print(len(xy))

30268


In [10]:
x = xy.iloc[:,:-1]
y = xy.iloc[:,-1:]

In [11]:
from skmultiflow.evaluation import EvaluatePrequential
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [12]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [13]:
model = make_pipeline(StandardScaler(), SVC(kernel='rbf'))#SGDClassifier())

In [14]:
#window_size = 5
for i in range (1):
    model.fit(X_train, y_train.values.ravel())
    z = accuracy_score(y_test, model.predict(X_test))
    print(z)

0.9202730976764674


In [15]:
#save model
#fname = input()
with open('SVM_Activity_model_window5_new_all.pkl','wb') as f:
    pickle.dump(model,f)

In [27]:
#load+use model
with open('SVM_Activity_model.pkl', 'rb') as f:
    clf = pickle.load(f)

#clf.predict(XXX)

# APPENDIX

In [18]:
df = pd.read_csv('dataset/running.csv')
df

,time,x,y,z
0,2:24:18 AM,-2.545114,-14.061576,-1.620924
1,2:24:18 AM,0.373507,-12.481355,-2.643279
2,2:24:18 AM,5.985687,-16.091920,-5.964138
3,2:24:18 AM,6.598621,-14.880418,-6.706364
4,2:24:18 AM,4.654470,-11.147743,-2.413429
...,...,...,...,...
395,2:24:25 AM,0.804476,-20.942722,-3.342408
396,2:24:25 AM,1.316851,-14.518883,-4.309695
397,2:24:25 AM,2.844399,-12.213196,-3.828445
398,2:24:25 AM,1.834015,-12.045596,-2.865947


In [13]:
df.sample(frac=1, axis=0)#.reset_index()

,time,x1,x2,x3
117,2:24:20 AM,-3.423813,-19.434328,-1.441353
357,2:24:25 AM,-2.188367,-20.499780,-3.926611
22,2:24:18 AM,10.735929,-13.982565,12.258687
162,2:24:21 AM,1.929785,-7.015225,1.103761
373,2:24:25 AM,-3.878725,-2.253013,0.612934
...,...,...,...,...
180,2:24:21 AM,-8.281796,-8.387145,-0.311256
114,2:24:20 AM,-1.448536,-15.249136,-4.213923
144,2:24:20 AM,0.581809,-5.655277,-1.824437
315,2:24:24 AM,-1.604164,-19.683332,-5.190788


In [61]:
df.iloc[:,1].to_numpy()

array([-2.54511400e+00,  3.73506870e-01,  5.98568730e+00,  6.59862140e+00,
        4.65447040e+00,  2.51638300e+00, -5.02797700e-02, -9.40950000e-01,
       -1.83640870e+00, -1.58022140e+00, -1.83640870e+00, -3.58422950e+00,
       -3.47169850e+00, -4.87953200e+00, -5.93540700e+00, -1.02474970e+01,
       -1.43536780e+01, -9.99849100e+00, -3.47409270e+00, -3.04791200e+00,
        1.03193250e+00,  4.64249900e+00,  1.07359290e+01,  2.66482780e+00,
        1.34797670e+00,  1.17319460e+00,  8.90670240e-01,  1.69993500e-01,
       -4.88432050e-01, -7.11099600e-01, -1.05587520e+00, -2.92819800e+00,
       -4.46053400e+00, -6.76861500e+00, -8.86360550e+00, -8.57150400e+00,
       -8.65769800e+00, -8.08546600e+00, -4.03914170e+00, -5.18360500e+00,
       -2.57863400e+00, -7.90110700e-02,  7.82688430e+00,  1.19234890e+01,
        4.48208240e+00,  2.61454820e+00, -1.27614840e+00, -3.21790530e+00,
       -3.47409270e+00, -3.50282400e+00, -1.85077440e+00, -1.15164610e+00,
        0.00000000e+00, -

In [36]:
x1 = df.x1.to_numpy()
x2 = df.x2.to_numpy()
x3 = df.x3.to_numpy()

In [39]:
x1_tran = window(x1)
x2_tran = window(x2)
x3_tran = window(x3)

In [40]:
print(len(x1_tran))
x1_tran

396


array([[-2.545114  ,  0.37350687,  5.9856873 ,  6.5986214 ,  4.6544704 ],
       [ 0.37350687,  5.9856873 ,  6.5986214 ,  4.6544704 ,  2.516383  ],
       [ 5.9856873 ,  6.5986214 ,  4.6544704 ,  2.516383  , -0.05027977],
       ...,
       [-7.5898514 , -1.5275474 ,  0.8044763 ,  1.3168511 ,  2.8443985 ],
       [-1.5275474 ,  0.8044763 ,  1.3168511 ,  2.8443985 ,  1.8340145 ],
       [ 0.8044763 ,  1.3168511 ,  2.8443985 ,  1.8340145 ,  0.2561874 ]])

In [18]:
len(window(x1))

396

In [29]:
x1_tran

array([[-2.545114  ,  0.37350687,  5.9856873 ,  6.5986214 ,  4.6544704 ],
       [ 0.37350687,  5.9856873 ,  6.5986214 ,  4.6544704 ,  2.516383  ],
       [ 5.9856873 ,  6.5986214 ,  4.6544704 ,  2.516383  , -0.05027977],
       ...,
       [-7.5898514 , -1.5275474 ,  0.8044763 ,  1.3168511 ,  2.8443985 ],
       [-1.5275474 ,  0.8044763 ,  1.3168511 ,  2.8443985 ,  1.8340145 ],
       [ 0.8044763 ,  1.3168511 ,  2.8443985 ,  1.8340145 ,  0.2561874 ]])

In [48]:
table1 = pd.DataFrame(data=x1_tran)
table2 = pd.DataFrame(data=x2_tran)
table3 = pd.DataFrame(data=x3_tran)

In [53]:
#making df_window
table = pd.concat([table1, table2, table3], axis=1)
table.columns = [i for i in range (15)]

In [54]:
table

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-2.545114,0.373507,5.985687,6.598621,4.654470,-14.061576,-12.481355,-16.091920,-14.880418,-11.147743,-1.620924,-2.643279,-5.964138,-6.706364,-2.413429
1,0.373507,5.985687,6.598621,4.654470,2.516383,-12.481355,-16.091920,-14.880418,-11.147743,-7.130150,-2.643279,-5.964138,-6.706364,-2.413429,-0.332804
2,5.985687,6.598621,4.654470,2.516383,-0.050280,-16.091920,-14.880418,-11.147743,-7.130150,-4.278569,-5.964138,-6.706364,-2.413429,-0.332804,-1.551490
3,6.598621,4.654470,2.516383,-0.050280,-0.940950,-14.880418,-11.147743,-7.130150,-4.278569,-3.150866,-6.706364,-2.413429,-0.332804,-1.551490,-3.589018
4,4.654470,2.516383,-0.050280,-0.940950,-1.836409,-11.147743,-7.130150,-4.278569,-3.150866,-2.013585,-2.413429,-0.332804,-1.551490,-3.589018,-4.077450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,-3.117346,-4.240261,-7.589851,-1.527547,0.804476,-3.840417,-3.845205,-7.130150,-17.011322,-20.942722,0.265765,-0.059857,-0.610540,-1.254600,-3.342408
392,-4.240261,-7.589851,-1.527547,0.804476,1.316851,-3.845205,-7.130150,-17.011322,-20.942722,-14.518883,-0.059857,-0.610540,-1.254600,-3.342408,-4.309695
393,-7.589851,-1.527547,0.804476,1.316851,2.844399,-7.130150,-17.011322,-20.942722,-14.518883,-12.213196,-0.610540,-1.254600,-3.342408,-4.309695,-3.828445
394,-1.527547,0.804476,1.316851,2.844399,1.834015,-17.011322,-20.942722,-14.518883,-12.213196,-12.045596,-1.254600,-3.342408,-4.309695,-3.828445,-2.865947
